# 5.Основы парсинга и работы с API

In [2]:
import pandas as pd
from numpy import NaN
import requests
from bs4 import BeautifulSoup
import time

### Задание 1.
<b>Обязательная часть</b>

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

<i>KEYWORDS = [‘python’, ‘парсинг’]</i>

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

<b>Дополнительная часть (необязательная)</b>

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [16]:
def add_full_text(link):
  headers = {
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'
  }
  url = link
  req = requests.get(url = url, headers = headers).text
  soup = BeautifulSoup(req)
  full_ = soup.find('div', {"id":"post-content-body"}).text
  return full_

def find_posts():
  url = 'https://habr.com/ru/all/'
  keywords = ['python', 'парсинг']
  headers = {
      'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.9',
      'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'
  }
  req = requests.get(url = url, headers = headers).text
  soup = BeautifulSoup(req)
  news = soup.find_all('article', {"class": "tm-articles-list__item"})
  habr_blog = pd.DataFrame()

  for el in news:
    date = el.find('span', class_= 'tm-article-snippet__datetime-published').find('time').get('datetime')
    title = el.find('h2').text
    link = el.find('h2').find('a').get('href')
    text = add_full_text('https://habr.com' + link)
    row = {'date': date, 'title': title, 'link': link, 'text': text}

    for s in keywords:
      if s.lower() in text.lower():
        habr_blog = pd.concat([habr_blog, pd.DataFrame([row])])

  return habr_blog.reset_index(drop=True)

In [17]:
find_posts()

,date,title,link,text
0,2022-03-30T10:40:04.000Z,Под капотом МойОфис: кое-что про импортозамеще...,/ru/post/658171/,Тяп-ляп и кпдв на хабрDisclaimer:1. Сразу хочу...
1,2022-03-30T10:37:10.000Z,Защита веб-приложения в 2022: что должен уметь...,/ru/company/pentestit/blog/657899/,Интернет - опасное место. Мы регулярно слышим ...
2,2022-03-30T09:00:01.000Z,Георадар — как способ увидеть «подземный мир»,/ru/company/ruvds/blog/657661/,"\r\nЧеловек всегда мечтал знать о том, что нах..."
3,2022-03-30T08:48:43.000Z,"Импорт и экспорт данных в PostgreSQL, гайд для...",/ru/post/658153/,В процессе обучения аналитике данных у человек...
4,2022-03-30T08:43:44.000Z,"Ключевые метрики: как мы рассчитывали RPS, а п...",/ru/company/koshelek/blog/657857/,"Привет! Меня зовут Даня, и я дата-аналитик в К..."
5,2022-03-30T08:28:32.000Z,Как мы «завели» десятки команд в один кластер ...,/ru/company/jetinfosystems/blog/658147/,У OpenSearch неоднозначная репутация. Некоторы...
6,2022-03-30T08:28:32.000Z,Как мы «завели» десятки команд в один кластер ...,/ru/company/jetinfosystems/blog/658147/,У OpenSearch неоднозначная репутация. Некоторы...


### Задание 1 (обновлённое)
Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>
В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.

In [12]:
def webscr():
    url = 'https://habr.com/ru/search/'
    keywords = [
      'python', 
      'анализ данных'
    ]
    df_list = []
    for keyword in keywords:
      res = requests.get(url, params={'q':[keyword]})
      soup = BeautifulSoup(res.text)
      articles = soup.find_all('article', class_='tm-articles-list__item')
      blogs = pd.DataFrame()
      for article in articles:
        title = article.find('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2')
        if title:
          title = title.text
          link = article.find('a', class_='tm-article-snippet__title-link').get('href')
          date = article.find('span', class_='tm-article-snippet__datetime-published').text
        else:
          title = NaN
          link = NaN
          date = NaN
        df_list.append(pd.DataFrame({'title': [title],'date': [date], 'link': [link]}))
      blogs = pd.concat(df_list)
      blogs = blogs.drop_duplicates()
    return blogs.reset_index(drop=True).dropna(subset=['title'])

webscr()

,title,date,link
0,Жаждущим автоматизации: открытый урок «ChatOps...,13 декабря 2021 в 09:00,/ru/company/southbridge/news/t/595093/
1,Курс «Python для инженеров». Старт 3 потока 31...,20 января в 18:37,/ru/company/southbridge/news/t/646825/
2,"Вышел Python 2.7.18, последний релиз ветки Pyt...",21 апреля 2020 в 18:35,/ru/news/t/498364/
3,Python Community Meetup 8/07: видео и материал...,6 июля 2021 в 13:29,/ru/company/raiffeisenbank/news/t/566370/
4,Открытый урок «Пишем Custom Prometheus Exporte...,13 января в 18:35,/ru/company/southbridge/news/t/645485/
5,Python как компилируемый статически типизирова...,4 декабря 2020 в 21:03,/ru/news/t/531402/
6,Вышел мартовский релиз расширения Python для V...,8 марта в 12:13,/ru/news/t/654707/
7,... начале этого года Python сместил Java и ...,3 марта 2020 в 13:22,/ru/company/itsumma/news/t/490834/
8,EPAM разработала бесплатный курс по программир...,16 ноября 2021 в 16:09,/ru/company/epam_systems/news/t/589555/
9,Вышла версия 0.0.2 snakeware — дистрибутива Li...,3 июня 2020 в 09:00,/ru/news/t/505096/


In [11]:
def get_habr_search_results(queries):
    url = 'https://habr.com/ru/search/'
    headers = {
    'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Mobile Safari/537.36'
    }
    habr_search_results = pd.DataFrame()
    for query in queries:
        req = requests.get(url, params= {'q':[query]}, headers=headers).text
        time.sleep(0.33)
        soup = BeautifulSoup(req)
        news = soup.find_all('article', class_='tm-articles-list__item')
        for el in news:
            try:
                title = el.find('h2', class_='tm-article-snippet__title').text
                date = el.find('span', class_='tm-article-snippet__datetime-published')
                link = 'https://habr.com' + el.find('a', class_='tm-article-snippet__title-link').get('href')
            
            except AttributeError:
                title = el.find('h2', class_='tm-megapost-snippet__title').text
                date = el.find('time', class_='tm-megapost-snippet__datetime-published')
                link = 'https://habr.com' + el.find('a', class_='tm-megapost-snippet__link tm-megapost-snippet__card').get('href')
            row = {'date': date, 'title': title, 'link': link}
            habr_search_results = pd.concat([habr_search_results, pd.DataFrame([row])])
    return habr_search_results.reset_index(drop=True)
get_habr_search_results(['анализ данных', 'sql'])

,date,title,link
0,[[25 июля 2012 в 13:29]],Пять лет Школе анализа данных,https://habr.com/ru/company/yandex/blog/148443/
1,[[25 июля 2017 в 13:47]],10 лет Школе анализа данных Яндекса,https://habr.com/ru/company/yandex/blog/334066/
2,[[14 января 2021 в 10:00]],Что такое бессерверный SQL? И как использовать...,https://habr.com/ru/company/microsoft/blog/537...
3,[[21 января 2019 в 11:21]],"Как мы строим систему обработки, хранения и ан...",https://habr.com/ru/company/sibur_official/blo...
4,[[14 мая 2020 в 13:39]],Ubic создаст платформу для анализа данных о мо...,https://habr.com/ru/news/t/501944/
5,[[16 апреля 2020 в 07:10]],Microsoft анонсировала запуск «Планетарного ко...,https://habr.com/ru/news/t/497474/
6,[[13 ноября 2021 в 19:15]],Владимир Путин выступил с Германом Грефом на к...,https://habr.com/ru/news/t/588991/
7,[[10 декабря 2021 в 13:15]],Интенсив для повышения квалификации: как испол...,https://habr.com/ru/company/netologyru/news/t/...
8,[[18 октября 2021 в 15:08]],AI Journey Contest 2021: какие задачи мы подго...,https://habr.com/ru/company/sbercloud/news/t/5...
9,[[18 июня 2014 в 15:19]],Обзор наиболее интересных материалов по анализ...,https://habr.com/ru/post/226641/


### Задание 2.

<b>Обязательная часть</b>

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Check. Список email-ов задаем переменной в начале кода:

<i>EMAIL = [xxx@x.ru, yyy@y.com]</i>

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

<b><small>Подсказка: сервис работает при помощи “скрытого” API. Внимательно изучите post-запросы.</small></b>

In [6]:
def leaks():
  url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
  payload = {"emailAddresses": ['xxx@x.ru', 'yyy@y.com']}
  headers = {
      'Vaar-Header-App-Build-Version': '1.0.0',
      'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
      'Vaar-Version': '0'
      }
  req = requests.post(url = url, json = payload, headers = headers)
  df = pd.DataFrame(req.json()['breaches']).T.reset_index()
  return df[['publishDate', 'site', 'description']]

In [7]:
leaks()

,publishDate,site,description
0,2020-01-03T00:00:00Z,azcentral.com,"At an unconfirmed date, online Arizona newspap..."
1,2021-02-11T00:00:00Z,forums.vkmonline.com,"At an unconfirmed date, the Russian-language m..."
2,2016-10-24T00:00:00Z,dropbox.com,Cloud storage company Dropbox suffered a major...
3,2019-10-17T00:00:00Z,zynga.com,"In September 2019, the game developer Zynga wa..."
4,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
5,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
6,2018-02-18T00:00:00Z,netlog.com,Netlog (formerly known as Facebox and Bingbox)...
7,2017-03-15T00:00:00Z,globalreach.eu,"In 2016, Global Reach Technology's database wa..."
8,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
9,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
